In [ ]:
import os
import datetime
import math
from time import time
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from PIL import Image
from matplotlib.pyplot import imshow

In [ ]:
print(tf.__version__)

2.9.2


In [ ]:
# Augmentation and model
IMG_SIZE            = 256
IMG_CHANNELS        = 1
BATCH_SIZE          = 32
PREFETCH_N          = 4
SHUFFLE_BUFFER_SIZE = 32
TRAIN_PERCENT       = 80
TEST_PERCENT        = 10
VAL_PERCENT         = 10

dataset = tf.data.experimental.load("drive/MyDrive/dataset_kacper.tf")
dataset2 = tf.data.experimental.load("drive/MyDrive/dataset_kacper_extend.tf")
dataset = dataset.concatenate(dataset2)
NR_OF_IMAGES        = len(dataset)
NR_OF_TRAIN_IMAGES  = math.floor(NR_OF_IMAGES*(TRAIN_PERCENT/100))
NR_OF_TEST_IMAGES   = math.ceil(NR_OF_IMAGES*(TEST_PERCENT/100))
NR_OF_VAL_IMAGES    = math.ceil(NR_OF_IMAGES*(VAL_PERCENT/100))
 
# Training
EPOCHS = 40
STEPS_PER_EPOCH = NR_OF_TRAIN_IMAGES // BATCH_SIZE

#Logs and callback
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
checkpoint_filepath = 'checkpoint/a/{epoch}'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='acc',
    mode='max',
    save_best_only=False)
stop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

In [ ]:
tf.data.experimental.save(dataset.take(NR_OF_TRAIN_IMAGES), "drive/MyDrive/dataset.zip", compression="GZIP")

In [ ]:
!zip -r /content/drive/MyDrive/dt.zip /content/drive/MyDrive/dataset.zip 

  adding: content/drive/MyDrive/dataset.zip/ (stored 0%)
  adding: content/drive/MyDrive/dataset.zip/dataset_spec.pb (deflated 9%)
  adding: content/drive/MyDrive/dataset.zip/11967020423983626880/ (stored 0%)
  adding: content/drive/MyDrive/dataset.zip/11967020423983626880/00000000.shard/ (stored 0%)
  adding: content/drive/MyDrive/dataset.zip/11967020423983626880/00000000.shard/00000000.snapshot (deflated 0%)
  adding: content/drive/MyDrive/dataset.zip/snapshot.metadata (stored 0%)


In [ ]:
def augment(image, label):
  #image = tf.image.random_crop(image, size=(224, 224, 1))
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  return image, label

def prepare_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image/255 - 0.5
  #image = tf.expand_dims(image, axis=-1)
  return image, label

def display_imgs(img, exp, pred):
    plt.figure(figsize=(15, 15))
    plt.title(f"Got: {pred}\nExpected: {exp}")
    plt.imshow(tf.keras.utils.array_to_img(img))
    plt.axis("off")
    plt.show()

In [ ]:
train = dataset.take(NR_OF_TRAIN_IMAGES)
valid = dataset.skip(NR_OF_TRAIN_IMAGES)
test = valid.take(NR_OF_TEST_IMAGES)
valid = valid.skip(NR_OF_TEST_IMAGES)

aug_train = train.map(prepare_image).map(augment).shuffle(1024).batch(BATCH_SIZE, drop_remainder=True).prefetch(PREFETCH_N)
valid = valid.map(prepare_image).batch(BATCH_SIZE, drop_remainder=True).prefetch(PREFETCH_N)
test  = test.map(prepare_image).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64,(3,3),activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128,(3,3),activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(32,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001),metrics=['accuracy'])

In [ ]:
model.fit(
    epochs=EPOCHS,
    x=aug_train,
    validation_data=valid,
    callbacks=[tensorboard_callback, model_checkpoint_callback, stop_callback]
)

results = []
for i in range(1, EPOCHS + 1):
  try:
    model.load_weights(f"checkpoint/a/{i}")
    result = model.evaluate(test)
    results.append(result[1])
  except:
    break

best = results.index(max(results)) + 1
print(f"Acc: {max(results)} Best epoch: {best}")
model.load_weights(f"checkpoint/a/{best}")
model.save('drive/MyDrive/model3.h5')

Epoch 1/40
2369/2369 [==============================] - 310s 125ms/step - loss: 0.7542 - accuracy: 0.6670 - val_loss: 0.5450 - val_accuracy: 0.7295
Epoch 2/40
2369/2369 [==============================] - 242s 102ms/step - loss: 0.4210 - accuracy: 0.7988 - val_loss: 0.3804 - val_accuracy: 0.8247
Epoch 3/40
2369/2369 [==============================] - 242s 102ms/step - loss: 0.3371 - accuracy: 0.8482 - val_loss: 0.2925 - val_accuracy: 0.8742
Epoch 4/40
2369/2369 [==============================] - 242s 102ms/step - loss: 0.2841 - accuracy: 0.8761 - val_loss: 0.2849 - val_accuracy: 0.8772
Epoch 5/40
2369/2369 [==============================] - 242s 102ms/step - loss: 0.2415 - accuracy: 0.8980 - val_loss: 0.2655 - val_accuracy: 0.8896
Epoch 6/40
2369/2369 [==============================] - 271s 114ms/step - loss: 0.2080 - accuracy: 0.9145 - val_loss: 0.2353 - val_accuracy: 0.9016
Epoch 7/40
2369/2369 [==============================] - 242s 102ms/step - loss: 0.1821 - accuracy: 0.9269 - val_